Terminal:

```
pip install google-api-python-client

pip install --upgrade google-api-python-client
```

In [33]:
from googleapiclient.discovery import build
from IPython.display import JSON
from pprint import pprint
import pandas as pd
import json

In [34]:
credentials_file_path = "./credentials.json"

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)


In [35]:
youtube = build('youtube', 'v3', developerKey=credentials['hb_youtube_api_key'])

In [36]:
def get_channel_stats(youtube, channel_ids):
    """
    Get channel statistics: title, subscriber count, view count, video count, upload playlist
    Params:
    
    youtube: the build object from googleapiclient.discovery
    channels_ids: list of channel IDs
    
    Returns:
    Dataframe containing the channel statistics for all channels in the provided list: title, subscriber count, view count, video count, upload playlist
    
    """
    all_data = []
    request = youtube.channels().list(
                part='snippet,contentDetails,statistics',
                id=','.join(channel_ids))
    response = request.execute() 
    
    for i in range(len(response['items'])):
        data = dict(channelName = response['items'][i]['snippet']['title'],
                    subscribers = response['items'][i]['statistics']['subscriberCount'],
                    views = response['items'][i]['statistics']['viewCount'],
                    totalVideos = response['items'][i]['statistics']['videoCount'],
                    playlistId = response['items'][i]['contentDetails']['relatedPlaylists']['uploads'])
        all_data.append(data)
    
    return pd.DataFrame(all_data)

def get_video_ids(youtube, playlist_id):
    """
    Get list of video IDs of all videos in the given playlist
    Params:
    
    youtube: the build object from googleapiclient.discovery
    playlist_id: playlist ID of the channel
    
    Returns:
    List of video IDs of all videos in the playlist
    
    """
    
    request = youtube.playlistItems().list(
                part='contentDetails',
                playlistId = playlist_id,
                maxResults = 50)
    response = request.execute()
    
    video_ids = []
    
    for i in range(len(response['items'])):
        video_ids.append(response['items'][i]['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    more_pages = True
    
    while more_pages:
        if next_page_token is None:
            more_pages = False
        else:
            request = youtube.playlistItems().list(
                        part='contentDetails',
                        playlistId = playlist_id,
                        maxResults = 50,
                        pageToken = next_page_token)
            response = request.execute()
    
            for i in range(len(response['items'])):
                video_ids.append(response['items'][i]['contentDetails']['videoId'])
            
            next_page_token = response.get('nextPageToken')
        
    return video_ids

def get_video_details(youtube, video_ids):
    """
    Get video statistics of all videos with given IDs
    Params:
    
    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs
    
    Returns:
    Dataframe with statistics of videos, i.e.:
        'channelTitle', 'title', 'description', 'tags', 'publishedAt'
        'viewCount', 'likeCount', 'favoriteCount', 'commentCount'
        'duration', 'definition', 'caption'
    """
        
    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
            
    return pd.DataFrame(all_video_info)

def get_comments_in_videos(youtube, video_ids):
    """
    Get top level comments as text from all videos with given IDs (only the first 10 comments due to quote limit of Youtube API)
    Params:
    
    youtube: the build object from googleapiclient.discovery
    video_ids: list of video IDs
    
    Returns:
    Dataframe with video IDs and associated top level comment in text.
    
    """
    all_comments = []
    
    for video_id in video_ids:
        try:   
            request = youtube.commentThreads().list(
                part="snippet,replies",
                videoId=video_id
            )
            response = request.execute()
        
            comments_in_video = [comment['snippet']['topLevelComment']['snippet']['textOriginal'] for comment in response['items'][0:10]]
            comments_in_video_info = {'video_id': video_id, 'comments': comments_in_video}

            all_comments.append(comments_in_video_info)
            
        except: 
            # When error occurs - most likely because comments are disabled on a video
            print('Could not get comments for video ' + video_id)
        
    return pd.DataFrame(all_comments)  


In [37]:
# Create the request object
data = []

search_response = youtube.search().list(
    part="snippet",
    maxResults=100,
    q="official music video",
    regionCode="US",
    type="video",
    videoCategoryId="10",
    fields="items(id/videoId,snippet(channelId,channelTitle,description,title)),nextPageToken,pageInfo,prevPageToken,regionCode"
)

# Execute the request and get the response
response = search_response.execute()

# Extract video IDs from the response
video_ids = [item['id']['videoId'] for item in response['items']]

for item in response['items']:
    video_id = item['id']['videoId']
    title = item['snippet']['title']
    channel_id = item['snippet']['channelId']
    channel_title = item['snippet']['channelTitle']
    description = item['snippet']['description']

    data.append({
        'video_id': video_id,
        'title': title,
        'channel_id': channel_id,
        'channel_title': channel_title,
        'description': description
    })

df = pd.DataFrame(data)
df.to_csv('music_videos_testdata.csv', index = False)

df

,video_id,title,channel_id,channel_title,description
0,XoiOOiuH8iI,Tyla - Water (Official Music Video),UCr2xjaS7CIx1OY7hYwrXb8A,TylaVEVO,Tyla - Water (Official Music Video) Listen To ...
1,suAR1PYFNYA,Dua Lipa - Houdini (Official Music Video),UC-J-KZfRV8c13fOCkhXdLiQ,Dua Lipa,"Limited edition Houdini CD and cassettes, this..."
2,UqyT8IEBkvY,Bruno Mars - 24K Magic (Official Music Video),UCoUM-UJ7rirJYP8CQ0EIaHA,Bruno Mars,"The official music video for Bruno Mars' ""24K ..."
3,SR6iYWJxHqs,Bruno Mars - Grenade (Official Music Video),UCoUM-UJ7rirJYP8CQ0EIaHA,Bruno Mars,"The official music video for ""Grenade"" by Brun..."
4,XzOvgu3GPwY,Taylor Swift ft. Ice Spice - Karma (Official M...,UCANLZYMidaCbLQFWXBC95Jg,TaylorSwiftVEVO,Listen to “Karma (feat. Ice Spice)” by Taylor ...
5,LHCob76kigA,Lukas Graham - 7 Years [Official Music Video],UCs8ynQgjoKZblUXosXoeOEQ,Lukas Graham,Watch the official music video for 7 Years by ...
6,5anLPw0Efmo,Evanescence - My Immortal (Official Music Video),UCDDmukrq0bwckrp09bNt-Rw,EvanescenceVEVO,"Watch the official music video for ""My Immorta..."
7,RlPNh_PBZb4,Olivia Rodrigo - vampire (Official Video),UCxE5jEls-T0QtlTHT8lI1lw,OliviaRodrigoVEVO,Listen to vampire out now: https://OliviaRodri...
8,kiB9qk4gnt4,Kodak Black - Super Gremlin [Official Music Vi...,UChEYVadfkMCfrKUi6qr3I1Q,Kodak Black,Kodak Black - Super Gremlin [Official Music Vi...
9,LjhCEhWiKXk,Bruno Mars - Just The Way You Are (Official Mu...,UCoUM-UJ7rirJYP8CQ0EIaHA,Bruno Mars,"The official music video for Bruno Mars' ""Just..."
